<a href="https://colab.research.google.com/github/DavidChoi76/2022_fall_nlp_do_it/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98_%EC%B2%AD%EC%99%80%EB%8C%80%EC%B2%AD%EC%9B%90_%EB%B2%A0%EC%9D%B4%EC%8A%A4%EB%9D%BC%EC%9D%B8_%EB%B9%88%EC%B9%B8%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

나와야 되는 것 : (4-2, 4-3에서 목록 만들자) 

- 패키지 로드하고, 인스톨
- 랜덤시드 고정 (= 같은 결과 재현 위해서)
- 로거 찍는거 <<
- 데이터 임포트
    - CSV → TXT 변환
- 학습용 말뭉치 (데이터셋) 선언
- 학습에 쓰일 인자들 + 각종 인자들을 미리 선언해줍니다
- 토크나이저
    - 데이터 클렌징은 되어 있었음
    - 만들고
    - 학습시키나
- 데이터 전처리 <<<
    - 자료형 변환. csv / txt → df
    - 구두점 제거
- 데이터 로더를 가져오기
- 프리트레인 된 모델 가져오기: 모델명, 인자값, 허깅페이스에서 가져옴
- 다운스트림 적용할 때 : 여기 들어가던가?????
- 평가 데이터 만들기
- 평가를 위한 코드????
- 태스크 정의
- 트레이너 만들고
- 학습



#1 환경설정 : 패키지 등

In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 453 kB 56.5 MB/s 


In [ ]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tqdm import tqdm

#2 데이터 불러오고, 전처리

In [ ]:
test = pd.read_csv('/content/gdrive/My Drive/cwcw/train.csv', encoding = 'utf-8')
train = pd.read_csv('/content/gdrive/My Drive/cwcw/test.csv', encoding = 'utf-8' )



In [ ]:
# 띄어쓰기 두개씩 된 것. 지워줘야 한다. 
import re
regex = r"[!\"#\$%&\'\(\)\*\+,-\./:;<=>\?@\[\\\]\^_`{\|}~]"
subst = ""

train = train.dropna()   # 데이터 중간 null 제거.(index ) -> pandas.dropna하기 
train_cleaned = []

for idx, row in train.iterrows():
  row.data = re.sub('n', '', row.data)      # 백스페이스 기호 지우기!
  row.data = re.sub("\s\s+" , " ", row.data) # 띄어쓰기 제거 
  row.data = re.sub(regex, subst, row.data)  # punctuation 제거!	
  train_cleaned.append(row.data) 
	#train.append(train_cleaned)
	#train_cleaned.concat([idx, row.data])

train["cleaned"] = train_cleaned # for문 밖에 있는 게 맞을 것 같아요

print(train.head(10))


   index                                               data  \
0      0  소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이\n법으로 인해 보호받지 못...   
1      1  국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에\n*지역 어린...   
2      2                   나경원파면. 나경원의원의  동계올림픽 위원을 파면해 주세요   
3      3  국민위원에가 삼성편만들어요. 삼성에서 11년간  일하고 혈암과 백혈병 진단을 받은 ...   
4      4  방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부...   
5      5  유은혜는 당장 사퇴하라!!!. 능력도 전문성도 없는 사람이 국회의윈 그리고 대통령의...   
6      6  신태용 감독. 노벨상 수상 청원합니다 !!!. 한국축구가 가장어려운 시기에,  용기...   
7      7  사회복무요원 최저임금 보장. 사회복무요원들은 의식주 보장이 아무것도 되질 않는데 왜...   
8      8  로또복권  의구심. 로또복귄 운영에  대한 민초들의  의구심을 해소하기 위하여 추첨...   
9      9  다자녀의기준이 뭘까요?. 오늘 상수도사업본부 감면신청을 보고 , 당황스러웠습니다. ...   

                                             cleaned  
0  소년법 폐지해주세요 법 아래에서 보호받아야 할 아이들이법으로 인해 보호받지 못하고 ...  
1  국공립 유치원 증설에 관하여 국공립 유치원 부지 학보와건립및 증설에지역 어린이 놀이...  
2                     나경원파면 나경원의원의 동계올림픽 위원을 파면해 주세요  
3  국민위원에가 삼성편만들어요 삼성에서 11년간 일하고 혈암과 백혈병 진단을 받은 사람...  
4  방과후유치원어린이집 영어교육을 유지시켜주세요 저는 

In [ ]:
print(train['data'][2818:2820])

2818    국민청원을 값어치 떨어지게한 이들을 벌받게 해주십시오.. 정말 힘든 사람들과 이야기...
2819    이영표를 축구 국대감독으로 추대. 대한민국 최고의 수비수  이영표를 대한민국 축구 ...
Name: data, dtype: object


   index                                               data
0      0  소년법 폐지해주세요. 법 아래에서 보호받아야 할 아이들이\n법으로 인해 보호받지 못...
1      1  국공립 유치원 증설에 관하여. 국공립 유치원 부지 학보와건립및 증설에\n*지역 어린...
2      2                   나경원파면. 나경원의원의  동계올림픽 위원을 파면해 주세요
3      3  국민위원에가 삼성편만들어요. 삼성에서 11년간  일하고 혈암과 백혈병 진단을 받은 ...
4      4  방과후,유치원,어린이집 영어교육을 유지시켜주세요. 저는 아이 셋 키우는 평범한 주부...
5      5  유은혜는 당장 사퇴하라!!!. 능력도 전문성도 없는 사람이 국회의윈 그리고 대통령의...
6      6  신태용 감독. 노벨상 수상 청원합니다 !!!. 한국축구가 가장어려운 시기에,  용기...
7      7  사회복무요원 최저임금 보장. 사회복무요원들은 의식주 보장이 아무것도 되질 않는데 왜...
8      8  로또복권  의구심. 로또복귄 운영에  대한 민초들의  의구심을 해소하기 위하여 추첨...
9      9  다자녀의기준이 뭘까요?. 오늘 상수도사업본부 감면신청을 보고 , 당황스러웠습니다. ...


#3 모델 생성, 학습, 예측
3-1 LSTM으로, 

3-2 Huggingface 모듈 불러와서 하는걸로 (처음으로 돌아가서, 패키지 로딩부터 다시)